In [1]:
import requests
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import pdist, squareform

In [2]:
# Define the API endpoint URL
url = "https://soilwise-he.containers.wur.nl/cat/collections/metadata:main/items"

# list to gather all data through API requests
soilwise_data_json = []

#configuration
limit = 50 # the max items the API provides is 50

# loop through pages to retreive next part of data
for i in range(0,20):
    print(f'loop num {i}')
    offset = (limit*i) # extract next 50 items
    params = {'limit':limit, 
              'offset': offset,
             } 
    headers = {"Accept": "application/json"}
    
    # Make the API request
    response = requests.get(url, headers=headers, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
         # Parse the JSON response
        soilwise_data_json = soilwise_data_json + data['features']
        print(f"id of first item extracted in this loop is {data['features'][0]['id']}")
    else:
        print(f"Failed to retrieve data: {response.status_code}")

soilwise_data_json[0]

loop num 0
id of first item extracted in this loop is 2e755407-6fab-4b3f-832d-fa76ea535ab0
loop num 1
id of first item extracted in this loop is 0b88bd89-2f3d-4005-ac15-2c8c0c6ef614
loop num 2
id of first item extracted in this loop is 16111ae4-69ba-477f-a6aa-b6fc4c9c5d58
loop num 3
id of first item extracted in this loop is 1f9cac1e-ac71-4007-bec1-bdf979fa4403
loop num 4
id of first item extracted in this loop is 2a3d1b59-350c-4db6-a28c-ed31407c1fa4
loop num 5
id of first item extracted in this loop is 362a01d3-3af1-400d-9436-840779995a4d
loop num 6
id of first item extracted in this loop is 417ff2b0-229d-421f-9c41-25e838090eed
loop num 7
id of first item extracted in this loop is 4dd7c40f-0fb9-4229-aaee-408d118856f0
loop num 8
id of first item extracted in this loop is 58d1aa0b-864a-4cf7-98dc-5915ba659aa8
loop num 9
id of first item extracted in this loop is 6409f4c7-0683-4471-b8f6-17b6dc8b2291
loop num 10
id of first item extracted in this loop is d49218ae-a95a-403f-b5b8-e927df26260

{'id': '2e755407-6fab-4b3f-832d-fa76ea535ab0',
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[9.14, 53.11],
    [9.14, 54.96],
    [10.77, 54.96],
    [10.77, 53.11],
    [9.14, 53.11]]]},
 'time': '2024-06-20',
 'properties': {'themes': [{'scheme': None, 'concepts': []},
   {'scheme': None, 'concepts': []}],
  'license': 'CC BY',
  'updated': '2024-06-20',
  'type': 'service',
  'created': '2020-03-06',
  'language': 'eng',
  'title': "WMS Service of the dataset 'V140 Kiel: Geographical reference and description of trial plots'",
  'description': "This WMS  Service includes spatial information used by datasets 'WMS Service of the dataset 'V140 Kiel: Geographical reference and description of trial plots''",
  'formats': ['CSV'],
  'keywords': ['infoMapAccessService',
   'Soil',
   'Organic amendments',
   'Geographical information systems',
   'Field experimentation',
   'Farm area',
   'data collection'],
  'providers': [{'name': 'Steffen Rothardt',
    'orga

In [3]:
print(f"total number of items extracted is {len(soilwise_data_json)}")

total number of items extracted is 1000


In [4]:
# Extract the relevant fields for each item
extracted_data = []
# iterate through the json and extract the fields needed for each item
for item in soilwise_data_json:
    extracted_item = {
        'id': item['id'],
        'title': item['properties'].get('title', ""), # Use an empty string if field is not available
        'description': item['properties'].get('description', ""), # Use an empty string if field is not available
        'keywords': item['properties'].get('keywords',[]) # Use an empty list if field is not available
    }
    extracted_data.append(extracted_item)

# Convert to DataFrame
soilwise_data_df = pd.DataFrame(extracted_data)

# checking for dupicates:
print(f"number of rows of dataframe is {soilwise_data_df.shape[0]}")
soilwise_data_df = soilwise_data_df.drop_duplicates(subset=['id'])
print(f"number of rows of dataframe after dropping duplicates is {soilwise_data_df.shape[0]}")


soilwise_data_df

number of rows of dataframe is 1000
number of rows of dataframe after dropping duplicates is 1000


,id,title,description,keywords
0,2e755407-6fab-4b3f-832d-fa76ea535ab0,WMS Service of the dataset 'V140 Kiel: Geograp...,This WMS Service includes spatial information...,"[infoMapAccessService, Soil, Organic amendment..."
1,0007bad6-848d-4763-9813-d5ed21cde6ee,Interactive effects of microplastics with othe...,Our study reveals the effects of GCFs on a soi...,"[Soil, microplastics, opendata, Multiple level..."
2,00682004-c6b9-4c1d-8b40-3afff8bbec69,SUSALPS temperature and volumetric soil water ...,Grassland is a precious good. Grassland contri...,"[environmental factors, water, Soil analysis, ..."
3,0086CC52-6F67-4393-99BE-7D3AB1B84160,Bodenübersichtskarte der Bundesrepublik Deutsc...,Die hier vorgestellte Bodenübersichtskarte im ...,"[Boden, Soil, Bodenart, Bodenauslaugung, Boden..."
4,166e8d03-d047-4031-ad12-113c0acb0f60,WMS Service of the dataset 'Impact of biopores...,This WMS Service includes spatial information ...,[infoMapAccessService]
...,...,...,...,...
995,dd49a3c4-38a2-4695-be36-72ff9303487c,Lysimeter data Rostock: P speciation of soil a...,The dataset contains P K-edge XANES data for s...,"[Soil, Phosphorus, Spectroscopy, Boden, P K-ed..."
996,dda3c8ac-7248-4ea1-a633-de15b15180b3,"Meteorological data, soil temperature and soil...",The data set was provided by the responsible p...,"[meteorological observations, conventional agr..."
997,dd41e401-ab56-40b2-bfa4-d82a899d972a,Web Map Service of the dataset 'Impact of diff...,This Web Map Service includes spatial informat...,"[infoMapAccessService, Soil, phosphorus, mycor..."
998,dd6a0c58-166a-4e79-82b0-f32051d14faa,Bee diversity in island-like habitats (kettle ...,"During June and July of 2017, wild bees were c...","[soil, opendata, pollination service, metacomm..."


In [5]:
# new columns to generate embeddings of original columns
soilwise_data_df['embedding_title'] = soilwise_data_df.title
soilwise_data_df['embedding_description'] = soilwise_data_df.description
soilwise_data_df['embedding_keywords'] = soilwise_data_df['keywords'].apply(lambda x: ', '.join(x))

soilwise_data_df

,id,title,description,keywords,embedding_title,embedding_description,embedding_keywords
0,2e755407-6fab-4b3f-832d-fa76ea535ab0,WMS Service of the dataset 'V140 Kiel: Geograp...,This WMS Service includes spatial information...,"[infoMapAccessService, Soil, Organic amendment...",WMS Service of the dataset 'V140 Kiel: Geograp...,This WMS Service includes spatial information...,"infoMapAccessService, Soil, Organic amendments..."
1,0007bad6-848d-4763-9813-d5ed21cde6ee,Interactive effects of microplastics with othe...,Our study reveals the effects of GCFs on a soi...,"[Soil, microplastics, opendata, Multiple level...",Interactive effects of microplastics with othe...,Our study reveals the effects of GCFs on a soi...,"Soil, microplastics, opendata, Multiple level;..."
2,00682004-c6b9-4c1d-8b40-3afff8bbec69,SUSALPS temperature and volumetric soil water ...,Grassland is a precious good. Grassland contri...,"[environmental factors, water, Soil analysis, ...",SUSALPS temperature and volumetric soil water ...,Grassland is a precious good. Grassland contri...,"environmental factors, water, Soil analysis, S..."
3,0086CC52-6F67-4393-99BE-7D3AB1B84160,Bodenübersichtskarte der Bundesrepublik Deutsc...,Die hier vorgestellte Bodenübersichtskarte im ...,"[Boden, Soil, Bodenart, Bodenauslaugung, Boden...",Bodenübersichtskarte der Bundesrepublik Deutsc...,Die hier vorgestellte Bodenübersichtskarte im ...,"Boden, Soil, Bodenart, Bodenauslaugung, Bodenb..."
4,166e8d03-d047-4031-ad12-113c0acb0f60,WMS Service of the dataset 'Impact of biopores...,This WMS Service includes spatial information ...,[infoMapAccessService],WMS Service of the dataset 'Impact of biopores...,This WMS Service includes spatial information ...,infoMapAccessService
...,...,...,...,...,...,...,...
995,dd49a3c4-38a2-4695-be36-72ff9303487c,Lysimeter data Rostock: P speciation of soil a...,The dataset contains P K-edge XANES data for s...,"[Soil, Phosphorus, Spectroscopy, Boden, P K-ed...",Lysimeter data Rostock: P speciation of soil a...,The dataset contains P K-edge XANES data for s...,"Soil, Phosphorus, Spectroscopy, Boden, P K-edg..."
996,dda3c8ac-7248-4ea1-a633-de15b15180b3,"Meteorological data, soil temperature and soil...",The data set was provided by the responsible p...,"[meteorological observations, conventional agr...","Meteorological data, soil temperature and soil...",The data set was provided by the responsible p...,"meteorological observations, conventional agri..."
997,dd41e401-ab56-40b2-bfa4-d82a899d972a,Web Map Service of the dataset 'Impact of diff...,This Web Map Service includes spatial informat...,"[infoMapAccessService, Soil, phosphorus, mycor...",Web Map Service of the dataset 'Impact of diff...,This Web Map Service includes spatial informat...,"infoMapAccessService, Soil, phosphorus, mycorr..."
998,dd6a0c58-166a-4e79-82b0-f32051d14faa,Bee diversity in island-like habitats (kettle ...,"During June and July of 2017, wild bees were c...","[soil, opendata, pollination service, metacomm...",Bee diversity in island-like habitats (kettle ...,"During June and July of 2017, wild bees were c...","soil, opendata, pollination service, metacommu..."


In [6]:
# multiple possibilities to convert text into contextual embeddings, current option is using transformers available via HugginFace platform (BERT, DISTILBERT, ... -> free),
# other options are to use contextual-richer LLM's (e.g. paying: GP or free LLama3 -> more resources needed! overkill, added value?)

# Set up with pre-trained model
model_name = 'distilbert-base-uncased'  # other models like 'bert-base-uncased' are also possible

# load tokenizer for selected model
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    raise

# load selected model
try:
    model = AutoModel.from_pretrained(model_name)
except Exception as e:
    print(f"Error loading model: {e}")
    raise

# Function to convert text into embeddings, each word is converted into vector and result is mean of all word-vectors. Other approaches are also possible, e.g. on sentence level, taking context of other words in sentence into account
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    # Take the mean of the token/word embeddings
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)
    return embeddings.detach().numpy()

# Apply the function to the DataFrame
soilwise_data_df['embedding_title'] = soilwise_data_df['embedding_title'].apply(get_embeddings)
soilwise_data_df['embedding_description'] = soilwise_data_df['embedding_description'].apply(get_embeddings)
soilwise_data_df['embedding_keywords'] = soilwise_data_df['embedding_keywords'].apply(get_embeddings)

soilwise_data_df

C:\Users\nberkvens\AppData\Local\anaconda3\envs\tensorflow_gee_3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,id,title,description,keywords,embedding_title,embedding_description,embedding_keywords
0,2e755407-6fab-4b3f-832d-fa76ea535ab0,WMS Service of the dataset 'V140 Kiel: Geograp...,This WMS Service includes spatial information...,"[infoMapAccessService, Soil, Organic amendment...","[[-0.26295176, -0.05457902, 0.18401955, -0.024...","[[-0.16696317, 0.023651835, 0.40922853, 0.0611...","[[-0.016777532, 0.19714673, 0.07908519, 0.2453..."
1,0007bad6-848d-4763-9813-d5ed21cde6ee,Interactive effects of microplastics with othe...,Our study reveals the effects of GCFs on a soi...,"[Soil, microplastics, opendata, Multiple level...","[[-0.033952005, 0.108728155, -0.06883621, 0.26...","[[-0.39372316, 0.16548389, 0.23728773, 0.02120...","[[-0.22446337, 0.1571082, 0.15740468, 0.091955..."
2,00682004-c6b9-4c1d-8b40-3afff8bbec69,SUSALPS temperature and volumetric soil water ...,Grassland is a precious good. Grassland contri...,"[environmental factors, water, Soil analysis, ...","[[-0.43514106, -0.17300248, 0.25394157, 0.0440...","[[-0.35968208, 0.10235999, 0.27212578, 0.02710...","[[-0.17351066, 0.57065666, 0.2568822, 0.093950..."
3,0086CC52-6F67-4393-99BE-7D3AB1B84160,Bodenübersichtskarte der Bundesrepublik Deutsc...,Die hier vorgestellte Bodenübersichtskarte im ...,"[Boden, Soil, Bodenart, Bodenauslaugung, Boden...","[[-0.107180506, 0.05131866, -0.004376183, -0.0...","[[-0.43826976, 0.0939387, 0.19622806, -0.09944...","[[-0.16472208, 0.16405283, 0.31736588, -0.0633..."
4,166e8d03-d047-4031-ad12-113c0acb0f60,WMS Service of the dataset 'Impact of biopores...,This WMS Service includes spatial information ...,[infoMapAccessService],"[[-0.30051106, 0.010823536, -0.00085008406, 0....","[[-0.17242524, 0.071774796, 0.23867568, 0.0861...","[[0.030550646, -0.16269122, 0.0698429, 0.25449..."
...,...,...,...,...,...,...,...
995,dd49a3c4-38a2-4695-be36-72ff9303487c,Lysimeter data Rostock: P speciation of soil a...,The dataset contains P K-edge XANES data for s...,"[Soil, Phosphorus, Spectroscopy, Boden, P K-ed...","[[-0.2660006, -0.0024695226, 0.1961377, 0.0865...","[[-0.3732139, 0.1163093, 0.19196309, 0.0178190...","[[-0.10512679, 0.29478765, 0.21147047, -0.0255..."
996,dda3c8ac-7248-4ea1-a633-de15b15180b3,"Meteorological data, soil temperature and soil...",The data set was provided by the responsible p...,"[meteorological observations, conventional agr...","[[-0.19836816, 0.18848011, 0.23790435, 0.06357...","[[-0.2669382, 0.03301047, 0.35249576, 0.100728...","[[-0.09898419, 0.348396, 0.3860768, 0.01230619..."
997,dd41e401-ab56-40b2-bfa4-d82a899d972a,Web Map Service of the dataset 'Impact of diff...,This Web Map Service includes spatial informat...,"[infoMapAccessService, Soil, phosphorus, mycor...","[[-0.28205073, 0.06666757, 0.0024443802, 0.111...","[[-0.22959174, 0.11016257, 0.07057317, 0.17223...","[[-0.14000942, 0.37072366, 0.2850479, 0.288238..."
998,dd6a0c58-166a-4e79-82b0-f32051d14faa,Bee diversity in island-like habitats (kettle ...,"During June and July of 2017, wild bees were c...","[soil, opendata, pollination service, metacomm...","[[-0.13129774, 0.066172116, -0.09157369, 0.034...","[[-0.18780026, 0.14625682, 0.07552566, 0.04365...","[[-0.09354025, 0.101906545, 0.131732, 0.037236..."


In [7]:
# Extract the list of embeddings
embeddings_title = np.vstack(soilwise_data_df['embedding_title'].values)
embeddings_description = np.vstack(soilwise_data_df['embedding_description'].values)
embeddings_keywords = np.vstack(soilwise_data_df['embedding_keywords'].values)

embeddings_title

array([[-0.26295176, -0.05457902,  0.18401955, ..., -0.15498509,
        -0.03641442, -0.01700304],
       [-0.03395201,  0.10872816, -0.06883621, ..., -0.39589286,
        -0.35345253,  0.07227551],
       [-0.43514106, -0.17300248,  0.25394157, ..., -0.16051476,
        -0.0865849 ,  0.22067128],
       ...,
       [-0.28205073,  0.06666757,  0.00244438, ..., -0.40080816,
        -0.00988611, -0.13360786],
       [-0.13129774,  0.06617212, -0.09157369, ..., -0.16610388,
        -0.11972532,  0.04935502],
       [-0.2100621 ,  0.04585158,  0.28211123, ..., -0.04859553,
         0.00249721, -0.04730187]], dtype=float32)

In [8]:
# multiple ways to determine similarity based on 'distance'/'direction' between vectors (Euclidean distance, cosine similarity, manhattan distance, jaccard similarity

# Calculate cosine similarity
sim_matrix_title = cosine_similarity(embeddings_title)
sim_matrix_description = cosine_similarity(embeddings_description)
sim_matrix_keywords = cosine_similarity(embeddings_keywords)

sim_matrix_title

array([[0.9999998 , 0.72755253, 0.8335133 , ..., 0.84243417, 0.8454625 ,
        0.72303736],
       [0.72755253, 0.99999976, 0.82303673, ..., 0.8568301 , 0.8264514 ,
        0.6822989 ],
       [0.8335133 , 0.82303673, 1.0000001 , ..., 0.8538825 , 0.818401  ,
        0.7596561 ],
       ...,
       [0.84243417, 0.8568301 , 0.8538825 , ..., 1.0000001 , 0.88914436,
        0.70687485],
       [0.8454625 , 0.8264514 , 0.818401  , ..., 0.88914436, 1.0000004 ,
        0.68818754],
       [0.72303736, 0.6822989 , 0.7596561 , ..., 0.70687485, 0.68818754,
        1.0000002 ]], dtype=float32)

In [9]:
# Convert the matrices to a DataFrame for better readability
sim_title_df = pd.DataFrame(sim_matrix_title, index=soilwise_data_df.rename(columns={'id':'id_1'}).id_1, columns=soilwise_data_df.rename(columns={'id':'id_2'}).id_2)
sim_description_df = pd.DataFrame(sim_matrix_description, index=soilwise_data_df.rename(columns={'id':'id_1'}).id_1, columns=soilwise_data_df.rename(columns={'id':'id_2'}).id_2)
sim_keywords_df = pd.DataFrame(sim_matrix_keywords, index=soilwise_data_df.rename(columns={'id':'id_1'}).id_1, columns=soilwise_data_df.rename(columns={'id':'id_2'}).id_2)

sim_title_df

id_2,2e755407-6fab-4b3f-832d-fa76ea535ab0,0007bad6-848d-4763-9813-d5ed21cde6ee,00682004-c6b9-4c1d-8b40-3afff8bbec69,0086CC52-6F67-4393-99BE-7D3AB1B84160,166e8d03-d047-4031-ad12-113c0acb0f60,3418ee89-8331-49a9-b3d2-e14b8fde88be,01da627d-9ad7-4414-9f32-222a139c2407,025b61e6-5e9b-4c03-9e48-fa22be02106f,00bee634-47e6-490b-89ba-2464c9f09c31,0160f3e2-aa36-431c-96f7-871dc41e5f8c,...,dc115e5b-b27a-494b-9054-21ecdbda8d9f,dc7d8d5f-3448-4f94-8fbd-14a4979a32ab,dcc61b52-3767-4d29-bcf0-26c5cec0afd0,dc95094f-5f65-433f-805d-16642da2616a,dd1948d5-525d-4705-851b-acd45f92eb8b,dd49a3c4-38a2-4695-be36-72ff9303487c,dda3c8ac-7248-4ea1-a633-de15b15180b3,dd41e401-ab56-40b2-bfa4-d82a899d972a,dd6a0c58-166a-4e79-82b0-f32051d14faa,de0ec281-9f57-44a5-b78e-5c60672d73fe
id_1,,,,,,,,,,,,,,,,,,,,,
2e755407-6fab-4b3f-832d-fa76ea535ab0,1.000000,0.727553,0.833513,0.752005,0.885135,0.924839,0.769051,0.813880,0.813501,0.863694,...,0.742557,0.507263,0.827588,0.844388,0.796979,0.861564,0.821779,0.842434,0.845463,0.723037
0007bad6-848d-4763-9813-d5ed21cde6ee,0.727553,1.000000,0.823037,0.617708,0.848116,0.722251,0.804379,0.693615,0.802788,0.851099,...,0.828225,0.518291,0.832834,0.800646,0.783742,0.810022,0.773859,0.856830,0.826451,0.682299
00682004-c6b9-4c1d-8b40-3afff8bbec69,0.833513,0.823037,1.000000,0.719942,0.866733,0.799670,0.822063,0.755467,0.840252,0.888579,...,0.815610,0.555473,0.802406,0.979098,0.822648,0.895643,0.855165,0.853882,0.818401,0.759656
0086CC52-6F67-4393-99BE-7D3AB1B84160,0.752005,0.617708,0.719942,1.000000,0.701170,0.718171,0.637447,0.706693,0.707506,0.718613,...,0.639714,0.494674,0.650903,0.744184,0.639187,0.697186,0.668543,0.684325,0.677488,0.775577
166e8d03-d047-4031-ad12-113c0acb0f60,0.885135,0.848116,0.866733,0.701170,1.000000,0.858879,0.834621,0.744966,0.862810,0.913519,...,0.827324,0.560622,0.837335,0.856243,0.822379,0.869020,0.831670,0.928040,0.893725,0.713759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dd49a3c4-38a2-4695-be36-72ff9303487c,0.861564,0.810022,0.895643,0.697186,0.869020,0.842087,0.823721,0.808952,0.855556,0.870155,...,0.837489,0.525811,0.843361,0.904782,0.863966,1.000000,0.843859,0.871039,0.846370,0.748114
dda3c8ac-7248-4ea1-a633-de15b15180b3,0.821779,0.773859,0.855165,0.668543,0.831670,0.764388,0.780935,0.740569,0.819321,0.851006,...,0.724773,0.494806,0.813475,0.871425,0.759831,0.843859,1.000000,0.811497,0.799589,0.680493
dd41e401-ab56-40b2-bfa4-d82a899d972a,0.842434,0.856830,0.853882,0.684325,0.928040,0.815296,0.837038,0.758880,0.905640,0.909556,...,0.866802,0.512407,0.844981,0.839624,0.823071,0.871039,0.811497,1.000000,0.889144,0.706875


In [10]:
# Stack the DataFrames and reset the index
stacked_sim_title_df = sim_title_df.stack().reset_index()
stacked_sim_description_df = sim_description_df.stack().reset_index()
stacked_sim_keywords_df = sim_keywords_df.stack().reset_index()
# Rename the columns
stacked_sim_title_df.columns = ['id_1','id_2','similarity_metric']
stacked_sim_description_df.columns = ['id_1','id_2','similarity_metric']
stacked_sim_keywords_df.columns = ['id_1','id_2','similarity_metric']
# Filter out self-similarity (similarity of items with themselves)
stacked_sim_title_df = stacked_sim_title_df[stacked_sim_title_df['id_1'] != stacked_sim_title_df['id_2']]
stacked_sim_description_df = stacked_sim_description_df[stacked_sim_description_df['id_1'] != stacked_sim_description_df['id_2']]
stacked_sim_keywords_df = stacked_sim_keywords_df[stacked_sim_keywords_df['id_1'] != stacked_sim_keywords_df['id_2']]
stacked_sim_title_df = stacked_sim_title_df.sort_values(by=['similarity_metric'], ascending=False)
stacked_sim_description_df = stacked_sim_description_df.sort_values(by=['similarity_metric'], ascending=False)
stacked_sim_keywords_df = stacked_sim_keywords_df.sort_values(by=['similarity_metric'], ascending=False)
stacked_sim_title_df

,id_1,id_2,similarity_metric
929221,c9a6def1-d330-475f-bf52-4931ae2b8bcf,2e48504f-4ae9-4400-b5fe-128b03863710,1.000000
221929,2e48504f-4ae9-4400-b5fe-128b03863710,c9a6def1-d330-475f-bf52-4931ae2b8bcf,1.000000
99097,500e7063-e828-4e82-b533-91c80a010817,551bd06c-9f14-4737-809d-32250a010852,1.000000
365477,5187f8c5-38ef-4b07-bc26-a5e257a8ef59,68749995-c4bf-4f80-94e5-43c2291c99be,1.000000
962958,d3e51946-2c46-4e5b-af86-800af4e21e3d,d36ad69a-42ed-42b8-be24-5492802fa876,1.000000
...,...,...,...
317477,44e6b58a-78fd-40bd-ac88-84974676bfca,68749995-c4bf-4f80-94e5-43c2291c99be,0.201143
590477,7fb4144f-f591-486f-b0db-ddad9f192932,68749995-c4bf-4f80-94e5-43c2291c99be,0.201026
365590,5187f8c5-38ef-4b07-bc26-a5e257a8ef59,7fb4144f-f591-486f-b0db-ddad9f192932,0.201026
477590,68749995-c4bf-4f80-94e5-43c2291c99be,7fb4144f-f591-486f-b0db-ddad9f192932,0.201026


In [11]:
# merge extra info about title etc.
# title
stacked_sim_title_df = stacked_sim_title_df.merge(soilwise_data_df[['id','title','description','keywords']], how='left', left_on='id_1', right_on='id')
stacked_sim_title_df = stacked_sim_title_df.merge(soilwise_data_df[['id','title','description','keywords']], how='left', left_on='id_2', right_on='id')
stacked_sim_title_df = stacked_sim_title_df.rename(columns={'title_x':'title_1','description_x':'description_1','keywords_x':'keywords_1','title_y':'title_2','description_y':'description_2','keywords_y':'keywords_2','similarity_metric':'similarity_metric_title'})
stacked_sim_title_df = stacked_sim_title_df[['similarity_metric_title','id_1','title_1','description_1','keywords_1','id_2','title_2','description_2','keywords_2']]
# description
stacked_sim_description_df = stacked_sim_description_df.merge(soilwise_data_df[['id']], how='left', left_on='id_1', right_on='id')
stacked_sim_description_df = stacked_sim_description_df.merge(soilwise_data_df[['id']], how='left', left_on='id_2', right_on='id')
stacked_sim_description_df = stacked_sim_description_df.rename(columns={'similarity_metric':'similarity_metric_description'})
stacked_sim_description_df = stacked_sim_description_df[['similarity_metric_description','id_1','id_2']]
# keywords
stacked_sim_keywords_df = stacked_sim_keywords_df.merge(soilwise_data_df[['id']], how='left', left_on='id_1', right_on='id')
stacked_sim_keywords_df = stacked_sim_keywords_df.merge(soilwise_data_df[['id']], how='left', left_on='id_2', right_on='id')
stacked_sim_keywords_df = stacked_sim_keywords_df.rename(columns={'similarity_metric':'similarity_metric_keywords'})
stacked_sim_keywords_df = stacked_sim_keywords_df[['similarity_metric_keywords','id_1','id_2']]

# merge all dataframes together
stacked_sim_df = pd.merge(left=stacked_sim_title_df, right=stacked_sim_description_df, how='left', left_on=['id_1', 'id_2'], right_on=['id_1', 'id_2'])
stacked_sim_df = pd.merge(left=stacked_sim_df, right=stacked_sim_keywords_df, how='left', left_on=['id_1', 'id_2'], right_on=['id_1', 'id_2'])
# rearrange
stacked_sim_df = stacked_sim_df[['similarity_metric_title','similarity_metric_description','similarity_metric_keywords','id_1','title_1','description_1','keywords_1','id_2','title_2','description_2','keywords_2']]

stacked_sim_df

,similarity_metric_title,similarity_metric_description,similarity_metric_keywords,id_1,title_1,description_1,keywords_1,id_2,title_2,description_2,keywords_2
0,1.000000,0.999369,0.788131,c9a6def1-d330-475f-bf52-4931ae2b8bcf,Proximal soil sensing data from the RapidMappe...,Proximal soil sensing data were collected by a...,"[Soil, soil pH, opendata, Proximal Soil Sensin...",2e48504f-4ae9-4400-b5fe-128b03863710,Proximal soil sensing data from the RapidMappe...,Proximal soil sensing data were collected by a...,"[Soil, opendata, Boden]"
1,1.000000,0.999369,0.788131,2e48504f-4ae9-4400-b5fe-128b03863710,Proximal soil sensing data from the RapidMappe...,Proximal soil sensing data were collected by a...,"[Soil, opendata, Boden]",c9a6def1-d330-475f-bf52-4931ae2b8bcf,Proximal soil sensing data from the RapidMappe...,Proximal soil sensing data were collected by a...,"[Soil, soil pH, opendata, Proximal Soil Sensin..."
2,1.000000,0.838298,0.968593,500e7063-e828-4e82-b533-91c80a010817,Půdní mapa 1 : 50 000,Mapy zobrazují půdní pokryv České republiky. V...,"[Půda, půdy, půda, typ půdní, mapa půdní, les,...",551bd06c-9f14-4737-809d-32250a010852,Půdní mapa 1 : 50 000,INSPIRE prohlížecí služba zpřístupňuje půdní m...,"[půdy, Půda, mapa půdní, půda, typ půdní, mapa..."
3,1.000000,1.000000,1.000000,5187f8c5-38ef-4b07-bc26-a5e257a8ef59,|,|,"[Boden, infoMapAccessService]",68749995-c4bf-4f80-94e5-43c2291c99be,|,|,"[Boden, infoMapAccessService]"
4,1.000000,0.774162,1.000000,d3e51946-2c46-4e5b-af86-800af4e21e3d,Sugar beet time series experiment (V465) for c...,This table contains observations of vegetation...,"[Soil, sugar beet, crop yield, irrigation, nit...",d36ad69a-42ed-42b8-be24-5492802fa876,Sugar beet time series experiment (V465) for c...,To create a holistic data set for the verifica...,"[Soil, sugar beet, crop yield, irrigation, nit..."
...,...,...,...,...,...,...,...,...,...,...,...
998995,0.201143,0.287283,0.707676,44e6b58a-78fd-40bd-ac88-84974676bfca,Abundance of different microorganism groups in...,The table contains the abundances of various b...,"[Soil, maize, field experimentation, soil micr...",68749995-c4bf-4f80-94e5-43c2291c99be,|,|,"[Boden, infoMapAccessService]"
998996,0.201026,0.270357,0.707676,7fb4144f-f591-486f-b0db-ddad9f192932,Abundance of different microorganism groups in...,The table contains the abundances of various b...,"[Soil, maize, field experimentation, soil micr...",68749995-c4bf-4f80-94e5-43c2291c99be,|,|,"[Boden, infoMapAccessService]"
998997,0.201026,0.270357,0.707676,5187f8c5-38ef-4b07-bc26-a5e257a8ef59,|,|,"[Boden, infoMapAccessService]",7fb4144f-f591-486f-b0db-ddad9f192932,Abundance of different microorganism groups in...,The table contains the abundances of various b...,"[Soil, maize, field experimentation, soil micr..."
998998,0.201026,0.270357,0.707676,68749995-c4bf-4f80-94e5-43c2291c99be,|,|,"[Boden, infoMapAccessService]",7fb4144f-f591-486f-b0db-ddad9f192932,Abundance of different microorganism groups in...,The table contains the abundances of various b...,"[Soil, maize, field experimentation, soil micr..."


In [12]:
# Create a new column with the maximum value of columns a, b, and c
stacked_sim_df['max_value'] = stacked_sim_df[['similarity_metric_title','similarity_metric_description','similarity_metric_keywords']].max(axis=1)

# Sort the DataFrame by the new column
# to minimalize the size of the dataframe, only select rows where at least one of the similarities is 1 
print(f'number of rows in dataframe is: {stacked_sim_df.shape[0]}')
stacked_sim_df = stacked_sim_df[stacked_sim_df.max_value == 1]
print(f'number of rows in dataframe only select rows where at least one of the similarities is 1: {stacked_sim_df.shape[0]}')
stacked_sim_df = stacked_sim_df.sort_values(by=['max_value','similarity_metric_title','similarity_metric_description','similarity_metric_keywords'], ascending=[False, False, False, False]).drop(columns='max_value')

stacked_sim_df

number of rows in dataframe is: 999000
number of rows in dataframe only select rows where at least one of the similarities is 1: 1182


,similarity_metric_title,similarity_metric_description,similarity_metric_keywords,id_1,title_1,description_1,keywords_1,id_2,title_2,description_2,keywords_2
3,1.000000,1.000000,1.000000,5187f8c5-38ef-4b07-bc26-a5e257a8ef59,|,|,"[Boden, infoMapAccessService]",68749995-c4bf-4f80-94e5-43c2291c99be,|,|,"[Boden, infoMapAccessService]"
8,1.000000,1.000000,1.000000,68749995-c4bf-4f80-94e5-43c2291c99be,|,|,"[Boden, infoMapAccessService]",5187f8c5-38ef-4b07-bc26-a5e257a8ef59,|,|,"[Boden, infoMapAccessService]"
6,1.000000,0.999322,1.000000,b8311ee8-4b8f-421a-aeac-2b4f0d4beff8,On-the-go gamma spectra for the site “Düren” f...,"The file contains 3,592 datasets. They compris...","[Soil, opendata, soil heterogeneity; proximal ...",7a740815-f3bf-461f-a8d3-b7e274d3aac2,On-the-go gamma spectra for the site “Düren” f...,The file contains 843 datasets. They comprise ...,"[Soil, opendata, soil heterogeneity; proximal ..."
11,1.000000,0.999322,1.000000,7a740815-f3bf-461f-a8d3-b7e274d3aac2,On-the-go gamma spectra for the site “Düren” f...,The file contains 843 datasets. They comprise ...,"[Soil, opendata, soil heterogeneity; proximal ...",b8311ee8-4b8f-421a-aeac-2b4f0d4beff8,On-the-go gamma spectra for the site “Düren” f...,"The file contains 3,592 datasets. They compris...","[Soil, opendata, soil heterogeneity; proximal ..."
7,1.000000,0.987513,0.970356,4b2b65c6-ff50-4669-99cc-ace343de3548,Scaling with ranked subsampling (SRS) algorith...,Scaling with ranked subsampling (SRS) is an al...,"[Soil, Bioinformatics, Computer programming, o...",9730f571-96c7-4490-b5c2-e99d4c63f572,Scaling with ranked subsampling (SRS) algorith...,--This data has been withdrawn by the author.-...,"[Bioinformatics, statistics, Statistical metho..."
...,...,...,...,...,...,...,...,...,...,...,...
594961,0.771850,0.914437,1.000000,68B4F032-A0CE-4B47-89C2-23DB80414102,INSPIRE - Annex III - Soil - Soil map (1:25.000),This dataset contains the Soil map of the Gran...,"[Soil, INSPIRE, high-value-dataset, National]",BD518FF7-C525-4250-B28C-BD4B8D884393,Soil acidity (pHCaCl2),Interpolation (Kriging) of the pH content (CaC...,"[Soil, INSPIRE, high-value-dataset, National]"
838076,0.691082,0.885626,1.000000,6B346278-D020-48BE-9617-CE8BA1513308,Topsoil Organic Carbon Content,Modeling of the content of topsoil organic car...,"[Soil, INSPIRE, high-value-dataset, National]",43ACDDFF-4F47-443E-B697-D36330486A75,INSPIRE - Annex III - Soil - Soil map (1:100.000),This dataset contains the Soil map of the Gran...,"[Soil, INSPIRE, high-value-dataset, National]"
838077,0.691082,0.885626,1.000000,43ACDDFF-4F47-443E-B697-D36330486A75,INSPIRE - Annex III - Soil - Soil map (1:100.000),This dataset contains the Soil map of the Gran...,"[Soil, INSPIRE, high-value-dataset, National]",6B346278-D020-48BE-9617-CE8BA1513308,Topsoil Organic Carbon Content,Modeling of the content of topsoil organic car...,"[Soil, INSPIRE, high-value-dataset, National]"
842448,0.688910,0.906683,1.000000,68B4F032-A0CE-4B47-89C2-23DB80414102,INSPIRE - Annex III - Soil - Soil map (1:25.000),This dataset contains the Soil map of the Gran...,"[Soil, INSPIRE, high-value-dataset, National]",6B346278-D020-48BE-9617-CE8BA1513308,Topsoil Organic Carbon Content,Modeling of the content of topsoil organic car...,"[Soil, INSPIRE, high-value-dataset, National]"


In [13]:
# Export the DataFrame to a CSV file
stacked_sim_df.to_csv('duplicate_datasheet_soilwise_cosinesim.csv', sep='\t', index=False)